# Part 1: Generate Hamiltonian, and check number of electrons

In [43]:
from min_part.ham_utils import obtain_OF_hamiltonian

from openfermion import (
    count_qubits,
    jordan_wigner,
)

from min_part.tensor_utils import get_chem_tensors, obt2op, tbt2op

mol = [["H", [0, 0, 0]], ["H", [0, 0, 0.8]]]
H, num_elecs = obtain_OF_hamiltonian(mol)
n_qubits = count_qubits(H)

In [44]:
from openfermion.linalg import qubit_operator_sparse

H_const, H_obt, H_tbt = get_chem_tensors(H=H, N=n_qubits)
H_ob_op = obt2op(H_obt)
H_tb_op = tbt2op(H_tbt)
H_ele = H_const + H_ob_op + H_tb_op

In [45]:
import pandas as pd
from dataclasses import dataclass
import scipy as sp
from min_part.operators import get_particle_number, get_total_spin, get_projected_spin

jw_of = jordan_wigner(H_ele)
jw_of_sp = qubit_operator_sparse(jw_of)
jw_op_array = jw_of_sp.toarray()
eigenvalues, eigenvectors = sp.linalg.eigh(jw_op_array)
eigenvalue_0 = eigenvalues[0]
eigenvectors_0 = eigenvectors[:, [0]]
eigenvectors_0_sparse = sp.sparse.csc_matrix(eigenvectors_0)

labels = ["Energy (Hartree)", "Occupied Spin Orbitals", "Total Spin", "Projected Spin"]
@dataclass
class EnergyOccupation:
    energy: float
    spin_orbs: int
    s_2: float
    s_z: float

def dc_to_dict(dcs, labels: list[str]):
    attrs = list(dcs[0].__dict__.keys())
    tb_dict = {}
    for i, label in enumerate(labels):
        tb_dict[label] = []
        for dc in dcs:
            tb_dict[label].append(getattr(dc, attrs[i]))
    return tb_dict

tb = []
for i in range(eigenvectors.shape[0]):
    w = eigenvectors[:, [i]]
    n = get_particle_number(w, 4)
    s_2 = get_total_spin(w, 2)
    s_z = get_projected_spin(w, 2)
    tb.append(EnergyOccupation(energy=eigenvalues[i], spin_orbs=n, s_2=s_2, s_z=s_z))

pd.DataFrame.from_dict(dc_to_dict(tb, labels=labels))

/Users/lucyhao/Codes/izmaylov-group/qc/min_part/operators.py:12: RuntimeWarning: invalid value encountered in divide
  n = np.divide(b, w)


,Energy (Hartree),Occupied Spin Orbitals,Total Spin,Projected Spin
0,-1.134148,2.0+0.0j,0.000000e+00+0.000000e+ 00j,0.0+0.0j
1,-0.597178,2.0+0.0j,2.000000e+00+0.000000e+ 00j,1.0+0.0j
2,-0.597178,2.0+0.0j,2.000000e+00+0.000000e+ 00j,0.0+0.0j
3,-0.597178,2.0+0.0j,2.000000e+00+0.000000e+ 00j,-1.0+0.0j
4,-0.556355,1.0+0.0j,7.500000e-01+0.000000e+ 00j,0.5+0.0j
5,-0.556355,1.0-0.0j,7.500000e-01-0.000000e+ 00j,-0.5-0.0j
6,-0.498232,3.0+0.0j,7.500000e-01+0.000000e+ 00j,0.5+0.0j
7,-0.498232,3.0+0.0j,7.500000e-01+0.000000e+ 00j,-0.5+0.0j
8,-0.227924,2.0-0.0j,-3.140185e-16-0.000000e+ 00j,-0.0-0.0j
9,0.151834,1.0+0.0j,7.500000e-01+0.000000e+ 00j,0.5+0.0j


In [46]:
from math import isclose
from min_part.utils import do_lr_fo

const, H1, H2_frags = do_lr_fo(H_ele)
dfs = []
allowed_two_elec_energies = []
allowed_gs_energies = []
all_energies = []
for frag in H2_frags:
    jw_of = jordan_wigner(frag)
    jw_of_sp = qubit_operator_sparse(jw_of)
    jw_op_array = jw_of_sp.toarray()
    eigenvalues, eigenvectors = sp.linalg.eigh(jw_op_array)
    tb = []
    two_elec_energies = []
    gs_energies = []
    all_en = []
    for i in range(eigenvectors.shape[0]):
        energy = eigenvalues[i]
        w = eigenvectors[:, [i]]
        n = get_particle_number(w, 4)
        s_2 = get_total_spin(w, 2)
        s_z = get_projected_spin(w, 2)
        tb.append(EnergyOccupation(energy=energy, spin_orbs=n, s_2=s_2, s_z=s_z))
        all_en.append(energy)
        if n == 2:
            two_elec_energies.append(energy)
            if isclose(s_2, 0,abs_tol=1e-6) and isclose(s_z, 0, abs_tol=1e-6):
                gs_energies.append(energy)
    all_energies.append(all_en)
    allowed_two_elec_energies.append(two_elec_energies)
    allowed_gs_energies.append(gs_energies)
    dfs.append(pd.DataFrame.from_dict(dc_to_dict(tb, labels=labels)))

/Users/lucyhao/Codes/izmaylov-group/qc/min_part/operators.py:12: RuntimeWarning: invalid value encountered in divide
  n = np.divide(b, w)
/Users/lucyhao/Codes/izmaylov-group/qc/min_part/operators.py:12: RuntimeWarning: divide by zero encountered in divide
  n = np.divide(b, w)
/var/folders/_g/6lsfhb794rzg7hm54031db2r0000gn/T/ipykernel_8976/228173179.py:28: ComplexWarning: Casting complex values to real discards the imaginary part
  if isclose(s_2, 0,abs_tol=1e-10) and isclose(s_z, 0, abs_tol=1e-10):


In [47]:
dfs[0]

,Energy (Hartree),Occupied Spin Orbitals,Total Spin,Projected Spin
0,-8.673617e-19,0.0+0.0j,0.00+0.00j,0.0+0.0j
1,1.732982e-06,2.0+0.0j,1.00+0.00j,0.0+0.0j
2,1.732982e-06,2.0+0.0j,2.00+0.00j,1.0+0.0j
3,1.732982e-06,2.0+0.0j,2.00+0.00j,-1.0+0.0j
4,1.732982e-06,2.0+0.0j,1.00+0.00j,0.0+0.0j
5,6.931927e-06,4.0+0.0j,0.00+0.00j,0.0+0.0j
6,5.092389e-03,3.0+0.0j,0.75+0.00j,-0.5+0.0j
7,5.092389e-03,3.0+0.0j,0.75+0.00j,0.5+0.0j
8,5.282005e-03,1.0+0.0j,0.75+0.00j,-0.5+0.0j
9,5.282005e-03,1.0+0.0j,0.75+0.00j,0.5+0.0j


In [48]:
dfs[1]

,Energy (Hartree),Occupied Spin Orbitals,Total Spin,Projected Spin
0,0.000000e+00,0.0+0.0j,0.000000e+00+0.000000e+ 00j,0.0+0.0j
1,0.000000e+00,2.0+0.0j,2.000000e+00+0.000000e+ 00j,-1.0+0.0j
2,6.938894e-18,2.0+0.0j,2.000000e+00+0.000000e+ 00j,1.0+0.0j
3,6.938894e-18,4.0+0.0j,0.000000e+00+0.000000e+ 00j,0.0+0.0j
4,2.220446e-16,2.0+0.0j,2.000000e+00+0.000000e+ 00j,0.0+0.0j
5,2.220446e-16,2.0+0.0j,0.000000e+00+0.000000e+ 00j,0.0+0.0j
6,9.231339e-02,1.0+0.0j,7.500000e-01+0.000000e+ 00j,0.5+0.0j
7,9.231339e-02,3.0+0.0j,7.500000e-01+0.000000e+ 00j,-0.5+0.0j
8,9.231339e-02,1.0+0.0j,7.500000e-01+0.000000e+ 00j,0.5+0.0j
9,9.231339e-02,3.0+0.0j,7.500000e-01+0.000000e+ 00j,-0.5+0.0j


In [49]:
dfs[2]

,Energy (Hartree),Occupied Spin Orbitals,Total Spin,Projected Spin
0,-2.775558e-16,0.0+0.0j,0.00+0.00j,0.0+0.0j
1,3.261900e-01,1.0+0.0j,0.75+0.00j,-0.5+0.0j
2,3.261900e-01,1.0+0.0j,0.75+0.00j,0.5+0.0j
3,3.381138e-01,1.0+0.0j,0.75+0.00j,-0.5+0.0j
4,3.381138e-01,1.0+0.0j,0.75+0.00j,0.5+0.0j
5,1.304760e+00,2.0+0.0j,0.00+0.00j,0.0+0.0j
6,1.328500e+00,2.0+0.0j,1.00+0.00j,0.0+0.0j
7,1.328500e+00,2.0+0.0j,2.00+0.00j,-1.0+0.0j
8,1.328500e+00,2.0+0.0j,2.00+0.00j,1.0+0.0j
9,1.328500e+00,2.0+0.0j,1.00+0.00j,0.0+0.0j


In [50]:
H_no_two_body = const + H1
jw_of = jordan_wigner(H_no_two_body)
jw_of_sp = qubit_operator_sparse(jw_of)
jw_op_array = jw_of_sp.toarray()
eigenvalues, eigenvectors = sp.linalg.eigh(jw_op_array)
energy_no_two_body = eigenvalues[0]
eigenvectors_0 = eigenvectors[:, [0]]
eigenvectors_0_sparse = sp.sparse.csc_matrix(eigenvectors_0)
two_body_contributions_gs_constraints =  sum([min(e, default=0) for e in allowed_gs_energies])
two_body_contributions = sum([min(e) for e in allowed_two_elec_energies])
two_body_contributions_not_filtered = sum([min(e) for e in all_energies])
print(f"Two body contribution with only the same GS states: {two_body_contributions_gs_constraints} Hartree")
print(f"Two body contribution with only two occupied spin orbitals states: {two_body_contributions} Hartree")
print(f"Two body contribution with any spin orbitals states: {two_body_contributions_not_filtered} Hartree")

print(f"LR Energy using only same GS symmetry orbital states: {energy_no_two_body + two_body_contributions_gs_constraints} Hartree")
print(f"LR Energy using only two occupied spin orbital states: {energy_no_two_body + two_body_contributions} Hartree")
print(f"LR Energy using only any spin orbital states: {energy_no_two_body + two_body_contributions_not_filtered} Hartree")


Two body contribution with only the same GS states: 1.3258879694558316 Hartree
Two body contribution with only two occupied spin orbitals states: 1.3047616839038485 Hartree
Two body contribution with any spin orbitals states: -2.7842311789427754e-16 Hartree
LR Energy using only same GS symmetry orbital states: -3.186943577424553 Hartree
LR Energy using only two occupied spin orbital states: -3.2080698629765365 Hartree
LR Energy using only any spin orbital states: -4.512831546880385 Hartree
